# import

In [1]:
## import functions from .py utils file: 
from Utilities.preprocessing import *
import os
from IPython.display import display, HTML
from tqdm import tqdm 



# paths

In [2]:
InputPath = r"Z:\projects\sequence_squad\revision_data\lars_recordings\bpod_data\\"
# InputPath = r"D:\EPHYS_BPOD_data_example\\"

out_path_base = r"Z:\projects\sequence_squad\revision_data\organised_data\animals\\"

Replace = True



# set paths

In [19]:

# find main paths for the animals
main_paths = []
Animal_ID = []
Animal_save_paths = []
Animal_bpod_save_paths = []
Animal_ephys_dates = []
for mouse in os.listdir(InputPath):
    if not mouse == 'FakeSubject':
        Animal_ID += [mouse]
        main_paths+=[os.path.join(InputPath,mouse,r'Sequence_Automated','Session Data//')]
        
        # find dates that match an ephys recording 
        for animal_file in os.listdir(out_path_base):
            # for seq00 animals
            if mouse.lower() in animal_file:
                recordings = os.listdir(os.path.join(out_path_base,animal_file))
                ephys_dates = list(recording.split('_')[-1] for recording in recordings)
                save_paths = list(os.path.join(out_path_base,animal_file,recording) for recording in recordings)
                ## extract date from recordings
            elif 'ap5' in mouse.lower():
                if 'lr' in mouse.lower():
                    recordings = os.listdir(os.path.join(out_path_base,'ap5lr_implant1'))
                elif 'L' in mouse:
                    recordings = os.listdir(os.path.join(out_path_base,'ap5L_implant1'))
                elif 'R' in mouse:
                    recordings = os.listdir(os.path.join(out_path_base,'ap5R_implant1'))
                ephys_dates = list(recording.split('_')[-1] for recording in recordings)
                save_paths = list(os.path.join(out_path_base,animal_file,recording) for recording in recordings)
                

        # create full save paths    
        full_save_paths = []
        for save_path in save_paths:
            new_path = os.path.join(save_path,r'behav_sync/2_task/')
            if not os.path.isdir(new_path):
                os.makedirs(new_path)
            full_save_paths+=[new_path]
        Animal_bpod_save_paths += [full_save_paths]
        Animal_save_paths += [save_paths]
        Animal_ephys_dates += [ephys_dates]
        
      


['18-11-2024', '19-11-2024']

In [ ]:
        
#### MAIN ####
for Animal_index, CurrentAnimal in enumerate(Animal_ID):
    display(HTML(f"<b>Processing data for: {CurrentAnimal}</b>"))
    
    #Load Behavioural data:
    Behav_data, Sessions,Path,FileDates = Import_EPHYS_Bpod_DataFiles(main_paths[Animal_index],Animal_ephys_dates[0])
    
    Processed = ('')
    Skipped = ('')

    for Session in tqdm(range(0, Sessions)):

        print(Session)

        filedate = FileDates[Session] + '_' + str(Behav_data[Session]['__header__'])[-25:-22]

        if Session < 10:
            Save_path = (Animal_save_paths[Animal_index][Session] + r'\\'+ r'\behav_sync\2_task\Preprocessed' + r'\\0' + str(Session) + '_' + filedate)
        else:
            Save_path = (Animal_save_paths[Animal_index][Session] + r'\\' + r'\behav_sync\2_task\Preprocessed' + r'\\' + str(Session) + '_' + filedate)
            
            
        print(Save_path)

        # see if dir exists already then use bool logic to detemrine if it needs to be processed or not
        Process = check_and_create_directory(Save_path, Replace)

        if Process == True:
        
            #Convert to python friendly format:
            convert_nested_structs(Behav_data[Session])

        #    # Extract GUI info
            Trial_settings = todict(Behav_data[Session]['SessionData']['TrialSettings'][0])
            FinalRewardAmount = []
            for item in Behav_data[Session]['SessionData']['SessionVariables']['TLevel']:
                TLevel = item
                FinalRewardAmount = FinalRewardAmount + [Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][4]]

            # save out training levels on their own
            filename = 'Preprocessed_TrainingLevels' 
            with open(Save_path + r'\\'+ filename, 'wb') as fp:
                pickle.dump(Behav_data[Session]['SessionData']['SessionVariables']['TLevel'], fp)

            # save out led intensites and reward amounts on their own:
            LED_Intensities = pd.DataFrame({'Port1':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port1'],
                                            'Port2':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port2'],
                                            'Port3':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port3'],
                                            'Port4':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port4'],
                                            'Port5':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port5']})
            LED_Intensities.to_csv(Save_path + '/Preprocessed_LED_Intensities.csv')
            RewardAmounts = pd.DataFrame({'Port1':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port1'],
                                           'Port2':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port2'],
                                           'Port3':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port3'],
                                           'Port4':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port4'],
                                           'Port5':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port5']})
            RewardAmounts.to_csv(Save_path + '/Preprocessed_RewardAmounts.csv')


            #Extract PortIn times for each port and check for errors (inside this function):
            All_PortIn_Times,All_PortOut_Times,All_Port_references = extract_poke_times(Behav_data[Session])

            #remove nans (times when part [in or out poke] of the event was dropped for some reason by bpod)
            All_PortIn_Times_fixed,All_PortOut_Times_fixed ,All_Port_references_fixed = remove_dropped_in_events(All_PortIn_Times,All_PortOut_Times,All_Port_references)
            
            # Resort these in time:
            All_PortIn_Times_sorted,All_PortOut_Times_sorted,All_Port_references_sorted = time_sort(All_PortIn_Times,All_PortOut_Times,All_Port_references)

            #extract reward times:
            Reward_ts = extract_reward_times(Behav_data[Session])
            
            # find reward inds and align rewarded ts to poke events:
            Rewarded_event_inds = find_reward_inds(All_PortIn_Times_sorted,All_Port_references_sorted,Reward_ts)
            Reward_ts = np.asarray(Reward_ts)
            Reward_ts = Reward_ts[np.logical_not(np.isnan(Reward_ts))]
            Reward_ts = list(Reward_ts)
            Reward_ts_aligned = align_trigger_to_index(Reward_ts,Rewarded_event_inds,All_Port_references_sorted)

            #extract trial start time stamps
            Trial_start_ts = extract_trial_timestamps(Behav_data[Session])

            #extract trial end times:
            Trial_end_ts = extract_trial_end_times(Behav_data[Session])

            #determine trial IDs
            trial_id = determine_trial_id(All_PortIn_Times_sorted,Trial_end_ts)

            # align trial start and end times to poke events
            trialstart_index = find_trialstart_index(trial_id)
            trial_start_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_start_ts)
            trial_end_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_end_ts)

            #determine LED and reward states for each trial and align them to trials:
            IntermediateRewards = []
            LED_intensities = []
            for item in Behav_data[Session]['SessionData']['SessionVariables']['TLevel']:
                TLevel = item
                IntermediateRewards = IntermediateRewards + [list(Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][0:4])]
                LED_intensities = LED_intensities + [list(Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][6:10])]
            aligned_LED_intensities = align_trial_start_end_timestamps(trial_id,trialstart_index,LED_intensities)
            aligned_IntermediateRewards = align_trial_start_end_timestamps(trial_id,trialstart_index,IntermediateRewards)


            Trial_start_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)
            Trial_end_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)
            First_poke_Camera_Ts_aligned = ['temporary_spacer'] * len(trial_id)

            ## align Training level for each trial:
            Training_Levels = align_opto_trials_to_dataframe(trial_id,Behav_data[Session]['SessionData']['SessionVariables']['TLevel'])

            # make portin dataframe:
            PortIn_df = pd.DataFrame(
                {'Trial_id' : trial_id,
                 'Trial_Start' : trial_start_ts_aligned,
                 'Port': All_Port_references_sorted,
                 'PokeIn_Time': All_PortIn_Times_sorted,
                 'PokeOut_Time': All_PortOut_Times_sorted,
                 'Reward_Times': Reward_ts_aligned,
                 'Trial_End' : trial_end_ts_aligned,
                 'Port 2,3,4,5 LED intensities': aligned_LED_intensities,
                 'Port 1,2,3,4 RewardAmount':aligned_IntermediateRewards,
                 'BACK_Trial_Start_Camera_Time' : Trial_start_Camera_Ts_aligned,
                 'BACK_Trial_End_Camera_Time' : Trial_end_Camera_Ts_aligned,
                 'BACK_First_poke_Camera_Time' : First_poke_Camera_Ts_aligned,
                'TrainingLevel': Training_Levels})

            #Save Data
            PortIn_df.to_csv(Save_path +'/PreProcessed_RawPokeData.csv')


AP5_1_L_Sequence_Automated_20241116_165741.mat
AP5_1_L_Sequence_Automated_20241118_152808.mat


KeyboardInterrupt: 

In [12]:
Save_path

'Z:\\projects\\sequence_squad\\revision_data\\organised_data\\animals\\\\seq008_implant1\\recording4_15-11-2024\\\\\\Preprocessed\\\\03_20241115_153541_Fri'